# We be stacking boxes

OK, so the elves are trying to unload boxes from the ship. Have a stack of boxes (designed with letters, see formatting section) and the directions that the crane operator is going to follow. We need to find out which boxes will be on top.

## Input formatting

It looks real weird. Here is the sample provided by the advent website:

```
    [D]    
[N] [C]    
[Z] [M] [P]
 1   2   3 

move 1 from 2 to 1
move 3 from 1 to 3
move 2 from 2 to 1
move 1 from 1 to 2
```
The actual input file has 9 stacks. Not entirely sure how I'm going to break down the this text file...

## The plan...
I believe I can split the stacks and the instructions on a `/n/n`. Instructions can then be split by `\n` and eventually rearranged so we end up with a dictionary for each line that looks like
```
{ "move": 1, "from": 2, "to": 1}
```
which we can then use to determine how many boxes we are moving from where/to. 

As for the boxes, that's going to be more complicated. We're gonna have to split on `\n` to get each line, then we can split lines by `' '`, I believe. After that I think we're going to start from the bottom up (tallest stack is 8 boxes) and add the `[A]` to another dictionary whose values are arrays of boxes:
```
{ "1": [ "[A]", "[G]" ], "2": [ "[J]", "[H]", "[C]" ] ... }
```
where the last value in the array is the box on top of the stack.

In [1]:
file = open('input.txt')
# split into boxes and instructions
data = file.read().split('\n\n')
box_data = data[0]
instruction_data = data[1]

In [2]:
box_data

'            [J] [Z] [G]            \n            [Z] [T] [S] [P] [R]    \n[R]         [Q] [V] [B] [G] [J]    \n[W] [W]     [N] [L] [V] [W] [C]    \n[F] [Q]     [T] [G] [C] [T] [T] [W]\n[H] [D] [W] [W] [H] [T] [R] [M] [B]\n[T] [G] [T] [R] [B] [P] [B] [G] [G]\n[S] [S] [B] [D] [F] [L] [Z] [N] [L]\n 1   2   3   4   5   6   7   8   9 '

In [3]:
instruction_data

'move 4 from 2 to 1\nmove 1 from 6 to 9\nmove 6 from 4 to 7\nmove 1 from 2 to 5\nmove 3 from 6 to 3\nmove 4 from 3 to 9\nmove 2 from 1 to 3\nmove 6 from 7 to 5\nmove 5 from 7 to 6\nmove 6 from 8 to 7\nmove 6 from 7 to 6\nmove 1 from 8 to 3\nmove 15 from 6 to 4\nmove 7 from 5 to 6\nmove 1 from 7 to 2\nmove 2 from 5 to 3\nmove 5 from 9 to 8\nmove 5 from 5 to 6\nmove 1 from 7 to 4\nmove 5 from 6 to 5\nmove 3 from 3 to 8\nmove 4 from 5 to 8\nmove 1 from 2 to 8\nmove 7 from 1 to 2\nmove 2 from 6 to 2\nmove 2 from 5 to 8\nmove 1 from 1 to 8\nmove 8 from 2 to 6\nmove 3 from 3 to 4\nmove 4 from 9 to 3\nmove 5 from 3 to 6\nmove 5 from 6 to 8\nmove 3 from 4 to 8\nmove 13 from 6 to 5\nmove 14 from 4 to 8\nmove 1 from 2 to 6\nmove 1 from 4 to 2\nmove 12 from 5 to 4\nmove 30 from 8 to 6\nmove 1 from 8 to 9\nmove 1 from 9 to 4\nmove 15 from 4 to 5\nmove 1 from 2 to 9\nmove 1 from 4 to 2\nmove 1 from 2 to 1\nmove 1 from 9 to 3\nmove 8 from 5 to 7\nmove 2 from 5 to 6\nmove 7 from 8 to 1\nmove 1 from 3

In [4]:
# gonna need to get rid of the last line
split_structs = instruction_data.split('\n')
instructions = split_structs[:(len(split_structs)-1)]

In [5]:
instructions

['move 4 from 2 to 1',
 'move 1 from 6 to 9',
 'move 6 from 4 to 7',
 'move 1 from 2 to 5',
 'move 3 from 6 to 3',
 'move 4 from 3 to 9',
 'move 2 from 1 to 3',
 'move 6 from 7 to 5',
 'move 5 from 7 to 6',
 'move 6 from 8 to 7',
 'move 6 from 7 to 6',
 'move 1 from 8 to 3',
 'move 15 from 6 to 4',
 'move 7 from 5 to 6',
 'move 1 from 7 to 2',
 'move 2 from 5 to 3',
 'move 5 from 9 to 8',
 'move 5 from 5 to 6',
 'move 1 from 7 to 4',
 'move 5 from 6 to 5',
 'move 3 from 3 to 8',
 'move 4 from 5 to 8',
 'move 1 from 2 to 8',
 'move 7 from 1 to 2',
 'move 2 from 6 to 2',
 'move 2 from 5 to 8',
 'move 1 from 1 to 8',
 'move 8 from 2 to 6',
 'move 3 from 3 to 4',
 'move 4 from 9 to 3',
 'move 5 from 3 to 6',
 'move 5 from 6 to 8',
 'move 3 from 4 to 8',
 'move 13 from 6 to 5',
 'move 14 from 4 to 8',
 'move 1 from 2 to 6',
 'move 1 from 4 to 2',
 'move 12 from 5 to 4',
 'move 30 from 8 to 6',
 'move 1 from 8 to 9',
 'move 1 from 9 to 4',
 'move 15 from 4 to 5',
 'move 1 from 2 to 9',
 'mov

In [6]:
split_boxes = box_data.split('\n')
split_boxes

['            [J] [Z] [G]            ',
 '            [Z] [T] [S] [P] [R]    ',
 '[R]         [Q] [V] [B] [G] [J]    ',
 '[W] [W]     [N] [L] [V] [W] [C]    ',
 '[F] [Q]     [T] [G] [C] [T] [T] [W]',
 '[H] [D] [W] [W] [H] [T] [R] [M] [B]',
 '[T] [G] [T] [R] [B] [P] [B] [G] [G]',
 '[S] [S] [B] [D] [F] [L] [Z] [N] [L]',
 ' 1   2   3   4   5   6   7   8   9 ']

In [7]:
split_boxes.pop()

' 1   2   3   4   5   6   7   8   9 '

In [8]:
split_boxes

['            [J] [Z] [G]            ',
 '            [Z] [T] [S] [P] [R]    ',
 '[R]         [Q] [V] [B] [G] [J]    ',
 '[W] [W]     [N] [L] [V] [W] [C]    ',
 '[F] [Q]     [T] [G] [C] [T] [T] [W]',
 '[H] [D] [W] [W] [H] [T] [R] [M] [B]',
 '[T] [G] [T] [R] [B] [P] [B] [G] [G]',
 '[S] [S] [B] [D] [F] [L] [Z] [N] [L]']

In [14]:
# not entirely sure how we're going to do this... let's test this
test_box_line = split_boxes[0].split(' ')

In [15]:
len(test_box_line)

27

In [16]:
test_box_line2 = split_boxes[1].split(' ')
print(len(test_box_line2))

21


In [17]:
test_box_line3 = split_boxes[7].split(' ')
print(len(test_box_line3))

9


In [20]:
# counting indexes: 1, 5, 9, 13, 17, 21, 25, 29, 33
box_idx = [1, 5, 9, 13, 17, 21, 25, 29, 33]
for i in box_idx:
    print(split_boxes[7][i])


S
S
B
D
F
L
Z
N
L


In [27]:
boxes = {
    "1": [],
    "2": [],
    "3": [],
    "4": [],
    "5": [],
    "6": [],
    "7": [],
    "8": [],
    "9": []
}
# starting from the bottom of the stack (idx 7)
stack_idx = 7
while stack_idx >= 0:
    stack = split_boxes[stack_idx]
    for i, box in enumerate(box_idx):
        # add the box (letter) to the correct number stack
        if not stack[box].isspace():
            boxes[str(i + 1)].append(stack[box])
    stack_idx -= 1
boxes

{'1': ['S', 'T', 'H', 'F', 'W', 'R'],
 '2': ['S', 'G', 'D', 'Q', 'W'],
 '3': ['B', 'T', 'W'],
 '4': ['D', 'R', 'W', 'T', 'N', 'Q', 'Z', 'J'],
 '5': ['F', 'B', 'H', 'G', 'L', 'V', 'T', 'Z'],
 '6': ['L', 'P', 'T', 'C', 'V', 'B', 'S', 'G'],
 '7': ['Z', 'B', 'R', 'T', 'W', 'G', 'P'],
 '8': ['N', 'G', 'M', 'T', 'C', 'J', 'R'],
 '9': ['L', 'G', 'B', 'W']}

## And now that that's done

Data is now ready for processing. In order to answer today's question we need to move the boxes in `boxes` around according to each line of our `instructions`. Each line in `instructions` will tell us how many boxes to move `instruction["move"]`, from which stack `instruction["from"]`, and where to `instruction["to"]`. `move` instructions will have to be a `for _ in range(i)`. `from` will utilize `pop` to remove  the last item from an array and then `append` the value to the `to` stack.

In [29]:
# move 3 from 4 to 7
def setup_instructions(line):
    split = line.split(' ')
    return { "move": int(split[1]), "from": int(split[3]), "to": int(split[5]) }

# { "move": 3, "from": 4, "to": 7}
setup_instructions('move 3 from 4 to 7')

{'move': 3, 'from': 4, 'to': 7}

In [35]:
move, frm, to = { "move": 3, "from": 4, "to": 7}.values()

In [36]:
frm

4

In [42]:
def execute_instruction(boxes, instruction):
    move, frm, to = instruction.values()
    for _ in range(move):
        box = boxes[str(frm)].pop()
        boxes[str(to)].append(box)
    return

In [43]:
test_structs = [ 
    'move 1 from 2 to 1',
    'move 3 from 1 to 3',
    'move 2 from 2 to 1',
    'move 1 from 1 to 2'
]
test_struct1 = { 'move': 1, 'from': 2, 'to': 1 }
test_struct2 = {'move': 3, 'from': 1, 'to': 3 }
test_boxes = {
    "1": ['Z', 'N'],
    "2": ['M', 'C', 'D'],
    "3": ['P']
}

# "1": ['Z', 'N', 'D'],
# "2": ['M', 'C'],
# "3": ['P']
execute_instruction(test_boxes, test_struct1)
print(test_box1)
print(test_boxes)

None
{'1': ['Z', 'N', 'D'], '2': ['M', 'C'], '3': ['P']}


In [46]:
# "1": [],
# "2": ['M', 'C'],
# "3": ['P', 'D', 'N', 'Z']
execute_instruction(test_boxes, test_struct1)
test_boxes

{'1': ['Z', 'N', 'D', 'C', 'M'], '2': [], '3': ['P']}

In [47]:
# resetting
test_boxes = {'1': ['Z', 'N', 'D'], '2': ['M', 'C'], '3': ['P'] }
execute_instruction(test_boxes, test_struct2)
test_boxes

{'1': [], '2': ['M', 'C'], '3': ['P', 'D', 'N', 'Z']}

In [49]:
def move_boxes(boxes, instructions):
    for line in instructions:
        instruction = setup_instructions(line)
        execute_instruction(boxes, instruction)
    return

# have to reset, this is why you really shouldn't alter original data
test_data = {
    "1": ['Z', 'N'],
    "2": ['M', 'C', 'D'],
    "3": ['P']
}
test_copy = test_data.copy()

# [C]
# [M]
# [P, D, N, Z]
move_boxes(test_copy, test_structs)
test_copy

{'1': ['C'], '2': ['M'], '3': ['P', 'D', 'N', 'Z']}

In [50]:
boxes_copy = boxes.copy()
move_boxes(boxes_copy, instructions)
boxes_copy

{'1': ['Z'],
 '2': ['G', 'T', 'S', 'W', 'T', 'D', 'Z', 'R', 'M', 'J', 'R'],
 '3': ['V', 'W', 'T', 'T', 'L'],
 '4': ['G', 'P', 'L', 'Q', 'J'],
 '5': ['Z', 'B', 'P', 'G', 'L', 'C', 'B', 'W', 'N', 'G'],
 '6': ['H', 'S'],
 '7': ['D',
  'W',
  'H',
  'B',
  'V',
  'W',
  'G',
  'R',
  'T',
  'B',
  'T',
  'F',
  'Q',
  'W',
  'F',
  'S',
  'N',
  'C'],
 '8': ['T'],
 '9': ['B', 'G', 'R']}

# Got it! Finally...

Answer based on above cell is `ZRLJGSCTR`. Now we find out that the boxes are going to be moved in a different manner because the crane is capable of picking up multiple boxes at once. So if the instructions say to move three boxes to another stack, the three retain their original order. So we're slicing arrays now. 

Using same instructions and same boxes, only need to update `execute_instructions` and `move_boxes`.

In [68]:
# the particular kind of crane that moves multiple boxes at once is version 9001 
def execute_9001_instruction(boxes, instruction):
    move, frm, to = instruction.values()
    moved_boxes = []
    for _ in range(move):
        box = boxes[str(frm)].pop()
        moved_boxes.append(box)
    moved_boxes.reverse()
    boxes[str(to)].extend(moved_boxes)
    return

In [69]:
# expected
# [Z, N]
# [M]
# [P, 'C', 'D']
test_data_again = {
    "1": ['Z', 'N'],
    "2": ['M', 'C', 'D'],
    "3": ['P']
}
execute_9001_instruction(test_data_again, {'move': 2, 'from': 2, 'to': 3})

In [70]:
test_data_again

{'1': ['Z', 'N'], '2': ['M'], '3': ['P', 'C', 'D']}

In [71]:
def move_boxes_pt2(boxes, instructions):
    for line in instructions:
        instruction = setup_instructions(line)
        execute_9001_instruction(boxes, instruction)
    return

test_data = {
    "1": ['Z', 'N'],
    "2": ['M', 'C', 'D'],
    "3": ['P']
}

# [M]
# [C]
# [P, Z, N, D]
move_boxes_pt2(test_data, test_structs)
test_data

{'1': ['M'], '2': ['C'], '3': ['P', 'Z', 'N', 'D']}

In [73]:
boxes_pt2 = {'1': ['S', 'T', 'H', 'F', 'W', 'R'],
 '2': ['S', 'G', 'D', 'Q', 'W'],
 '3': ['B', 'T', 'W'],
 '4': ['D', 'R', 'W', 'T', 'N', 'Q', 'Z', 'J'],
 '5': ['F', 'B', 'H', 'G', 'L', 'V', 'T', 'Z'],
 '6': ['L', 'P', 'T', 'C', 'V', 'B', 'S', 'G'],
 '7': ['Z', 'B', 'R', 'T', 'W', 'G', 'P'],
 '8': ['N', 'G', 'M', 'T', 'C', 'J', 'R'],
 '9': ['L', 'G', 'B', 'W']}
move_boxes_pt2(boxes_pt2, instructions)
boxes_pt2

{'1': ['P'],
 '2': ['S', 'B', 'H', 'W', 'G', 'L', 'H', 'Z', 'B', 'R', 'R'],
 '3': ['Z', 'M', 'C', 'J', 'T'],
 '4': ['L', 'W', 'G', 'D', 'T'],
 '5': ['T', 'J', 'N', 'B', 'T', 'S', 'W', 'W', 'N', 'G'],
 '6': ['G', 'R'],
 '7': ['V',
  'T',
  'W',
  'L',
  'D',
  'R',
  'Z',
  'Q',
  'B',
  'T',
  'F',
  'Q',
  'T',
  'C',
  'G',
  'S',
  'W',
  'F'],
 '8': ['P'],
 '9': ['V', 'G', 'B']}

# Got it. PRTTGRFPB